# FitFirst Study

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
from tqdm import tqdm, trange
from pyvis.network import Network
from coloring.fit_first import fit_first
from utils import generate_k_colorable_graph

### Specify parameters here

In [ ]:
N = 50    # Number of online k-colorable graphs to generate at each step

In [9]:
# k, n, avg_comp_ratio, N
study_results = []
vertex_range = range(0,1500,100)[::2]
# chromatic number
for k in [2,3,4]:
    # number of vertices
    for n in tqdm(vertex_range):
        ratios = []
        for _ in range(N):
            # Generate a k-colorable graph
            G, _ = generate_k_colorable_graph(k, n, 0.7)
            # Color the graph
            G_ = fit_first(G)
            # Number of unqiue colors used
            num_colors = len(set([G_.nodes[i]['group'] for i in G_.nodes]))
            # Ratio of colors used to chromatic number
            ratio = num_colors / k
            ratios.append(ratio)
        
        # Average ratio of colors used to chromatic number
        avg_ratio = np.mean(ratios)
        study_results.append((k, n, avg_ratio, N))

        if n % 1000 == 0:
            df = pd.DataFrame(study_results, columns=['k', 'n', 'avg_comp_ratio', 'N'])
            df.to_feather(f'../data/fit_first_study_checkpoint.feather')


 29%|██▊       | 2/7 [01:17<03:14, 38.85s/it]


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(study_results, columns=['k', 'n', 'avg_comp_ratio', 'N'])
df.to_feather('../data/fit_first_study.feather')